In [5]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

In [10]:
df_path = f"../WeatherPy/output/city_desc2.csv"
df = pd.read_csv(df_path).drop(columns = "Unnamed: 0")

In [11]:
df

,City Name,Temperature (F),Humidity %,CLoudiness %,Wind Speed (mph)
0,cayenne,81.05,81,89,17.81
1,cayenne,81.05,81,89,17.81
2,buta,69.53,97,100,1.43
3,uspenka,53.73,53,77,10.25
4,ercis,59.00,67,20,4.36
...,...,...,...,...,...
536,souillac,56.80,77,99,29.80
537,jamestown,58.75,57,79,16.44
538,cape town,-22.34,87,12,5.44
539,saint-paul,72.34,75,73,26.80
